In [1]:
import os

os.chdir('../../dvrk_simulator')
path = os.getcwd()

print(path)

import dvrk_simulator.vrep.vrep as vrep
import dvrk_simulator.vrep.ArmPSM as ArmPSM
import dvrk_simulator.vrep.simObjects as simObjects
import transforms3d

import numpy as np
import matplotlib.pyplot as plt

/home/arclab/vrep_dvrk_sim/dvrk_simulator


In [2]:
clientID = vrep.simxStart("172.17.0.1",19999,True,True,5000,5) # Connect to V-REP
res = vrep.simxSynchronous(clientID , True)
print(res)

0


In [3]:
vrep.simxStartSimulation(clientID, vrep.simx_opmode_blocking)
vrep.simxSynchronous(clientID, True)
vrep.simxSynchronousTrigger(clientID)
vrep.simxGetPingTime(clientID)

psm1 = ArmPSM.ArmPSM(armNumber=1, clientID=clientID)
psm1.getPoseAtEE(ignoreError = True, initialize=True)
psm1.getJointAngles(ignoreError=True, initialize=True)

(array([0., 0., 0., 0., 0., 0.]), 0.0)

In [4]:
left_cam = simObjects.camera(clientID=clientID, string_name="Vision_sensor_left")
right_cam = simObjects.camera(clientID=clientID, string_name="Vision_sensor_right")

def getStereoImagePairs():
    img_left  = np.fliplr(left_cam.getImage())
    img_right = np.fliplr(right_cam.getImage())
    
#     img_left[ np.where(img_left  == [0,0,0])] = 255
#     img_right[np.where(img_right == [0,0,0])] = 255

    return img_left, img_right

left_cam.getPoseAtHandle( psm1.base_handle,  left_cam.camera_handle,  initialize=True)
left_cam.getPoseAtHandle(left_cam.camera_handle, right_cam.camera_handle, initialize=True)

vrep.simxSynchronousTrigger(clientID)
vrep.simxGetPingTime(clientID)

def getCurrentHandEye():
    return left_cam.getPoseAtHandle( psm1.base_handle, left_cam.camera_handle,  initialize=False)

def vrep_to_transforms_quat(quat):
    return np.array([quat[3], quat[0], quat[1], quat[2]])


def transforms_to_vrep_quat(quat):
    return np.array([quat[1], quat[2], quat[3], quat[0]])

Failed to get position
1
Failed to get orientation
1
Failed to get position
1
Failed to get orientation
1


In [5]:
pos, quat = getCurrentHandEye()

axis_angle = transforms3d.quaternions.quat2axangle(vrep_to_transforms_quat(quat))
print("Hand eye is: ")
print("tvec = {}".format(pos*1000.0))
print("rvec = {}".format(axis_angle[0]*axis_angle[1]))

print("In matrix form hand-eye is:")
R = transforms3d.axangles.axangle2mat(axis_angle[0], axis_angle[1])
T = np.eye(4)
T[:3, :3] = R
T[:3, -1] = pos*1000.0

print("{}".format(T))
a_t = transforms3d.axangles.mat2axangle(R.transpose())
print("inverse tvec is = {}".format(np.linalg.inv(T)[:3,-1]))
print("inverse rvec is = {}".format(a_t[0]*a_t[1]))


pos, quat = left_cam.getPoseAtHandle(left_cam.camera_handle, right_cam.camera_handle, initialize=True)
print("Left camera to right camera transform is:")
print("T = {}".format(pos*1000.0))
print("R = {}".format(transforms3d.quaternions.quat2mat(vrep_to_transforms_quat(quat))))

Hand eye is: 
tvec = [140.60425758 -68.75953078  -3.63826752]
rvec = [ 0.43683098 -2.75821918  1.16153308]
In matrix form hand-eye is:
[[-9.51578403e-01 -3.07379641e-01  4.03731438e-03  1.40604258e+02]
 [-2.17668246e-01  6.64461846e-01 -7.14920268e-01 -6.87595308e+01]
 [ 2.17069294e-01 -6.81181482e-01 -6.99194329e-01 -3.63826752e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
inverse tvec is = [119.61896454  86.42865054 -52.26910179]
inverse rvec is = [-0.43683098  2.75821918 -1.16153308]
Left camera to right camera transform is:
T = [5.00004739 0.24092197 0.72932243]
R = [[ 1.00000000e+00  5.36441804e-07 -5.96046368e-08]
 [-5.36441802e-07  1.00000000e+00  2.98023384e-08]
 [ 5.96046528e-08 -2.98023064e-08  1.00000000e+00]]


In [6]:
from dvrk_simulator.vrep.vrepObject import vrepObject

class ArmECM(vrepObject):
    def __init__(self, clientID):

        super(ArmECM, self).__init__(clientID)
        self.base_handle = self.getHandle('L0_respondable_ECM')
        self.j1_handle   = self.getHandle('J1_ECM')
        self.j2_handle   = self.getHandle('J2_ECM')
        self.j3_handle   = self.getHandle('J3_ECM')
        self.j4_handle   = self.getHandle('J4_ECM')
        
        self.left_sensor = self.getHandle('Vision_sensor_left')
        
    def getJointAngles(self, ignoreError = False, initialize = False):
        pos1  = self.getJointPosition(self.j1_handle,  ignoreError, initialize)
        pos2  = self.getJointPosition(self.j2_handle,  ignoreError, initialize)
        pos3  = self.getJointPosition(self.j3_handle,  ignoreError, initialize)
        pos4  = self.getJointPosition(self.j4_handle,  ignoreError, initialize)
        
        return np.array([pos1, pos2, pos3, pos4])
    
    def setJointAngles(self, jointAngles, ignoreError = False):

        self.setJointPosition(self.j1_handle, jointAngles[0], ignoreError)
        self.setJointPosition(self.j2_handle, jointAngles[1], ignoreError)
        self.setJointPosition(self.j3_handle, jointAngles[2], ignoreError)
        self.setJointPosition(self.j4_handle, jointAngles[3], ignoreError)
        
    def getCurrentPose(self, ignoreError = False, initialize = False):
        return self.getPoseAtHandle(self.left_sensor, self.base_handle, ignoreError, initialize )
    
    
ecm = ArmECM(clientID=clientID)
ecm.getJointAngles(initialize=True)
ecm.getCurrentPose(initialize=True)

Failed to get joint angle
1
Failed to get joint angle
1
Failed to get joint angle
1
Failed to get joint angle
1
Failed to get position
1
Failed to get orientation
1


(array([0., 0., 0.]), array([0., 0., 0., 0.]))

In [7]:
import roslib
import rospy
import math
from sensor_msgs.msg import Image
from sensor_msgs.msg import JointState
from geometry_msgs.msg import PoseStamped
from cv_bridge import CvBridge
import cv2


rospy.init_node('vrep_simulation', anonymous=True)
l_image_pub = rospy.Publisher("/stereo/left/image",Image, queue_size=1)
r_image_pub = rospy.Publisher("/stereo/right/image",Image, queue_size=1)
j_publisher = rospy.Publisher("/vrep_sim/PSM1/joint_angles", JointState, queue_size=1)
j_ecm_publisher = rospy.Publisher("/vrep_sim/ECM/joint_angles", JointState, queue_size=1)
p_publisher = rospy.Publisher("/vrep_sim/ECM/position_cartesian_current", PoseStamped, queue_size=1)

bridge = CvBridge()


haveNewJTrackedData = False
haveNewPTrackedData = False
haveNewLTrackedData = False
haveNewRTrackedData = False

most_recent_tracked_joint = None
most_recent_tracked_pose = None
most_recent_tracked_limg = None
most_recent_tracked_rimg = None

def j_tracked_callback(j_msg):
    global most_recent_tracked_joint
    most_recent_tracked_joint = j_msg.position
    
    global haveNewJTrackedData
    haveNewJTrackedData = True


def pose_tracked_callback(p_msg):
    global most_recent_tracked_pose
    most_recent_tracked_pose = p_msg.pose
    
    global haveNewPTrackedData
    haveNewPTrackedData = True
    
def left_tracked_image_callback(l_msg):
    global most_recent_tracked_limg
    most_recent_tracked_limg = CvBridge().imgmsg_to_cv2(l_msg, desired_encoding='passthrough')
    
    global haveNewLTrackedData
    haveNewLTrackedData = True
    
def right_tracked_image_callback(r_msg):
    global most_recent_tracked_rimg
    most_recent_tracked_rimg = CvBridge().imgmsg_to_cv2(r_msg, desired_encoding='passthrough')

    global haveNewRTrackedData
    haveNewRTrackedData = True



j_tracked_sub     = rospy.Subscriber("/particle_filter/PSM1/state_joint_current", JointState, j_tracked_callback)
pose_tracked_sub  = rospy.Subscriber("/particle_filter/PSM1/position_cartesian_current", PoseStamped, pose_tracked_callback)
l_image_tracked_sub = rospy.Subscriber("/stereo/render/left/image", Image, left_tracked_image_callback)
r_image_tracked_sub = rospy.Subscriber("/stereo/render/right/image", Image, right_tracked_image_callback)




In [8]:
psm1.getJointAngles(ignoreError=True, initialize=True)


def getStretchedJointAngles(cableStretchPer90, cableStretchInsertionPerCM, cableStretchJawAngle):
    cableStretch_radians = cableStretchPer90*np.pi/180.0
    cableStretchInsertion_meters = cableStretchInsertionPerCM/100.0

    joint_angles, jaw_angle = psm1.getJointAngles(ignoreError=True, initialize=False)
    
    for i in range(0, len(joint_angles)):
        if(i == 2):
            joint_angles[i] = joint_angles[i]/(1.0 + cableStretchInsertion_meters/0.2 )
        else:
            joint_angles[i] = joint_angles[i]/(1.0 + cableStretch_radians/(np.pi/2.0))
        
    jaw_angle = jaw_angle/(1.0 + cableStretchJawAngle)
    
    return joint_angles, jaw_angle

def applyNoisyJointAnglesECM(joint_angles, std_rot, std_prism):    
    for i in range(0, len(joint_angles)):
        if(i == 2):
            joint_angles[i] += np.random.normal(scale=std_prism)
        else:
            joint_angles[i] += np.random.normal(scale=std_rot)
            
    return joint_angles

In [9]:
def randomRotation(max_rotation_about_axis = 0.05):
    # taken from second answer here: 
    # https://math.stackexchange.com/questions/442418/random-generation-of-rotation-matrices
    
    # "Uniformly sample a random axis" in the spherical space
    theta  = np.arccos(np.random.uniform(low=-1,high=1))
    phi    = np.random.uniform(low=0, high=2*np.pi)
    vector = np.array([np.sin(phi)*np.cos(theta), np.sin(phi)*np.sin(theta), np.cos(phi)])
    
    # Uniformly sample angle to rotate about
    angle  = np.random.uniform(low = 0, high = max_rotation_about_axis)
    return transforms3d.axangles.axangle2mat(angle=angle, axis=vector)

def applyRandomNoiseToPose(pos, quat, std_pos_noise, max_rotation_about_random_axis):
    pos += np.random.normal(loc=0, scale=std_pos_noise, size=(3,))
    
    randomR  = randomRotation(max_rotation_about_random_axis)
    currentR = np.dot(transforms3d.quaternions.quat2mat(vrep_to_transforms_quat(quat)), randomR)
    quat =  transforms_to_vrep_quat(transforms3d.quaternions.mat2quat(currentR))

    return pos, quat

def posQuatToMatrix(pos, quat):
    T = np.eye(4)
    T[:3,:3] = transforms3d.quaternions.quat2mat(quat)
    T[:3, 3] = pos
    return T

#Applies right-hand-side noise
def applyNormalNoiseToPose(pos, quat, std_pos_noise, std_aangle_noise):
    T_noise = np.eye(4)
    
    if(std_aangle_noise !=0):
        rvec = np.random.normal(loc=0, scale= std_aangle_noise, size=(3,))
        T_noise[:3,:3] = transforms3d.axangles.axangle2mat(rvec/np.linalg.norm(rvec), np.linalg.norm(rvec));
        
    T_noise[:3, 3] = np.random.normal(loc=0, scale=std_pos_noise, size=(3,))
    T_out = np.dot(posQuatToMatrix(pos, quat), T_noise)
    return T_out[:3,3], transforms3d.quaternions.mat2quat(T_out[:3,:3])

In [10]:
def writeHandEye(pos, quat, pos_ecm, quat_ecm, filePath):
    pos *=1000.0
    axis_angle = transforms3d.quaternions.quat2axangle(vrep_to_transforms_quat(quat))
    axis_angle = axis_angle[0]*axis_angle[1]
    
    
    pos_ecm *= 1000.0
    quat_ecm = vrep_to_transforms_quat(quat_ecm)
    axis_angle_ecm = transforms3d.quaternions.quat2axangle(quat_ecm)
    axis_angle_ecm = axis_angle_ecm[0]*axis_angle_ecm[1]
    

    with open(filePath, 'w+') as file:
        file.write("%YAML:1.0\n")
        file.write("---\n")
        file.write("PSM1_tvec: [{}, {}, {}]\n".format(pos[0], pos[1], pos[2]))
        file.write("PSM1_rvec: [{}, {}, {}]\n".format(axis_angle[0], axis_angle[1], axis_angle[2]))
        
        file.write("cam_tvec: [{}, {}, {}]\n".format(pos_ecm[0], pos_ecm[1], pos_ecm[2]))
        file.write("cam_rvec: [{}, {}, {}]\n".format(axis_angle_ecm[0], axis_angle_ecm[1], axis_angle_ecm[2]))

In [11]:
import os

def createDirectory(folder_name):
    if not os.path.exists(folder_name):
        os.mkdir(folder_name)

In [15]:
import subprocess
import time
import csv
import tf


e_pos, e_quat = psm1.getPoseAtHandle(psm1.EE_virtual_handle, left_cam.camera_handle,  initialize=True)


def runMovingArmAndECMExperiment(cableStretchPer90, cableStretchInsertionPerCM, cableStretchJawAngle, 
                                 abs_max_bias_rot, abs_max_bias_ins, abs_max_bias_jaw, 
                                 std_rot_ecm, std_prism_ecm,
                                 std_pos_noise, std_rot_noise, #for hand-eye noise
                                 hand_eye_filepath, save_folder):
    #save hand-eye
    starting_ecm_j_angles = [0., 0.2617994,  0., 0.]
    ecm.setJointAngles(starting_ecm_j_angles)

    vrep.simxSynchronousTrigger(clientID)
    vrep.simxGetPingTime(clientID)        
    vrep.simxSynchronousTrigger(clientID)
    vrep.simxGetPingTime(clientID)

    h_pos, h_quat = getCurrentHandEye()
    h_pos, h_quat = applyNormalNoiseToPose(h_pos, h_quat, std_pos_noise, std_rot_noise)
    ecm_pos, ecm_quat = ecm.getCurrentPose()
    writeHandEye(h_pos, h_quat, ecm_pos, ecm_quat, hand_eye_filepath)
    
    cmd = ["rosrun", "particle_filter", "particle_filter_node", 
           "/home/arclab/vrep_dvrk_sim/dvrk_simulator/vrep_sims_for_tool_tracking/vrep_dataset_configure_filter_with_ecm.json"]
    p = subprocess.Popen(cmd, stdout=subprocess.PIPE, stdin=subprocess.PIPE, stderr=subprocess.PIPE)
    time.sleep(4)
    
    std_of_random_position_traj = 0.0001 #in m, this is additive noise on trajectory
    max_rotation_about_random_axis_traj = 0.07 #in radians for trajectory, random walk!

    rate = rospy.Rate(100)

    #parameters for position trajectory
    xy_circle_amplitude = 0.025
    xy_circle_frequency = np.pi/180.0
    z_circle_amplitude = 0.01
    z_circle_frequency = np.pi/90.0
    jaw_rate = 1/25.0

    center_point  = [ 0.1405422 ,  0.03579695, -0.10973072];
    starting_quat = [-0.47013299, -0.26924748, -0.14768405,  0.827448  ]
    quat = starting_quat
    
    #Generate random bias values
    j_bias = np.random.uniform(-abs_max_bias_rot*np.pi/180.0, abs_max_bias_rot*np.pi/180.0, 2)
    j_bias = np.append(j_bias, np.random.uniform(-abs_max_bias_ins/100.0,abs_max_bias_ins/100.0,1))
    j_bias = np.append(j_bias, np.random.uniform(-abs_max_bias_rot*np.pi/180.0, abs_max_bias_rot*np.pi/180.0, 3))
    j_bias = np.append(j_bias, np.random.uniform(-abs_max_bias_jaw, abs_max_bias_jaw, 1))
    
    j_ecm_bias = np.random.uniform(-abs_max_bias_rot*np.pi/180.0, abs_max_bias_rot*np.pi/180.0, 2)
    j_ecm_bias = np.append(j_ecm_bias, np.random.uniform(-abs_max_bias_ins/100.0,abs_max_bias_ins/100.0,1))
    j_ecm_bias = np.append(j_ecm_bias, np.random.uniform(-abs_max_bias_rot*np.pi/180.0, abs_max_bias_rot*np.pi/180.0, 1))

    
    #CSV saving files
    createDirectory(save_folder)

    realHandeye_file = open(save_folder + "realHandeye.csv", 'wb')
    realHandeye_csv  = csv.writer(realHandeye_file, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
    
    realEndEffector_file = open(save_folder + "realEndEffector.csv", 'wb')
    realEndEffector_csv  = csv.writer(realEndEffector_file, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
    
    realJointAngles_file = open(save_folder + "realJointAngles.csv", 'wb')
    realJointAngles_csv  = csv.writer(realJointAngles_file, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)

    measuredJointAngles_file = open(save_folder + "measuredJointAngles.csv", 'wb')
    measuredJointAngles_csv  = csv.writer(measuredJointAngles_file, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
    
    trackedJointAngles_file = open(save_folder + "trackedJointAngles.csv", 'wb')
    trackedJointAngles_csv  = csv.writer(trackedJointAngles_file, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)

    trackedEndEffector_file = open(save_folder + "trackedEndEffector.csv", 'wb')
    trackedEndEffector_csv  = csv.writer(trackedEndEffector_file, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)

    trackedLumpedError_file = open(save_folder + "trackedLumpedError.csv", 'wb')
    trackedLumpedError_csv  = csv.writer(trackedLumpedError_file, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
    listener = tf.TransformListener()
    
    #ECM specific stuff!
    realECMPose_file = open(save_folder + "realECMPose.csv", 'wb')
    realECMPose_csv  = csv.writer(realECMPose_file, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)

    realJointAnglesECM_file = open(save_folder + "realECMJointAngles.csv", 'wb')
    realJointAnglesECM_csv  = csv.writer(realJointAnglesECM_file, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)

    measuredJointAnglesECM_file = open(save_folder + "measuredECMJointAngles.csv", 'wb')
    measuredJointAnglesECM_csv  = csv.writer(measuredJointAnglesECM_file, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)

    
    #Folders to save the images
    rawImageFilePath = save_folder + "rawImages/"
    createDirectory(rawImageFilePath)
    
    #Folders to save the images
    renderImageFilePath = save_folder + "renderImages/"
    createDirectory(renderImageFilePath)

    
    pose_header = ["Time Stamp", "x", "y", "z", "q_w", "q_x", "q_y", "q_z"]
    realHandeye_csv.writerow(pose_header)
    realEndEffector_csv.writerow(pose_header)
    trackedEndEffector_csv.writerow(pose_header)
    trackedLumpedError_csv.writerow(pose_header)
    realECMPose_csv.writerow(pose_header)
    
    joint_header = ["Time Stamp", "j1", "j2", "j3", "j4", "j5", "j6", "j7"]
    realJointAngles_csv.writerow(joint_header)
    measuredJointAngles_csv.writerow(joint_header)
    trackedJointAngles_csv.writerow(joint_header)
    realJointAnglesECM_csv.writerow(joint_header)
    measuredJointAnglesECM_csv.writerow(joint_header)
                                             
    start_itr = 0.0
    loop_itr  = 0.0
    
    firstTime =  True
    while not rospy.is_shutdown():

        #Do the trajectory
        x = xy_circle_amplitude*np.cos(xy_circle_frequency*loop_itr)
        y = xy_circle_amplitude*np.sin(xy_circle_frequency*loop_itr)
        z = z_circle_amplitude*np.sin(z_circle_frequency*loop_itr)

        pos  = center_point + np.array([x, y, z])
        pos, quat = applyRandomNoiseToPose(pos, quat, std_of_random_position_traj, max_rotation_about_random_axis_traj)

        #Move jaw!
        jaw  = math.fmod(jaw_rate*loop_itr, 2.0)
        if jaw > 1.0:
            jaw = 2.0 - jaw

        psm1.setPoseAtEE(pos, quat, 1 - jaw)
                                             
        #Do the trajectory for the ecm

        #first get the non-noisy pose which is published to the ros world later  
        ecm_j_angles = starting_ecm_j_angles + np.array([0.2*np.sin(2.0*xy_circle_frequency*loop_itr),
                                                         -0.1*np.sin(xy_circle_frequency*loop_itr),
                                                         -0.05*np.sin(xy_circle_frequency*loop_itr),
                                                         0.3*np.sin(xy_circle_frequency*loop_itr)])
                                             
        ecm.setJointAngles(ecm_j_angles)
        vrep.simxSynchronousTrigger(clientID)
        vrep.simxGetPingTime(clientID)        

        pos, quat = ecm.getCurrentPose()
        p_msg = PoseStamped()
        p_msg.pose.position.x = pos[0]
        p_msg.pose.position.y = pos[1]
        p_msg.pose.position.z = pos[2]
        p_msg.pose.orientation.x = quat[0]
        p_msg.pose.orientation.y = quat[1]
        p_msg.pose.orientation.z = quat[2]
        p_msg.pose.orientation.w = quat[3]

        #now set the noisy joint angles which give the image
        ecm_noisy_j_angles = applyNoisyJointAnglesECM(ecm_j_angles, std_rot_ecm, std_prism_ecm) + j_ecm_bias
        ecm.setJointAngles(ecm_noisy_j_angles)

        vrep.simxSynchronousTrigger(clientID)
        vrep.simxGetPingTime(clientID)   
        
                                                
        global haveNewJTrackedData
        global haveNewPTrackedData
        global haveNewLTrackedData
        global haveNewRTrackedData

        haveNewJTrackedData = False
        haveNewPTrackedData = False
        haveNewLTrackedData = False
        haveNewRTrackedData = False
        

        #Get timestamp and make the same for all data so it is sync'ed
        timestamp = rospy.get_rostime()

        #Publish images
        l_img, r_img = getStereoImagePairs()
        l_msg = bridge.cv2_to_imgmsg(l_img , encoding ="rgb8");
        l_msg.header.stamp = timestamp
        r_msg = bridge.cv2_to_imgmsg(r_img , encoding ="rgb8");
        r_msg.header.stamp = timestamp

        l_image_pub.publish(l_msg)
        r_image_pub.publish(r_msg)

        #Publish joint angles
        joint_angles, jaw_angle = getStretchedJointAngles(cableStretchPer90, 
                                                          cableStretchInsertionPerCM,
                                                          cableStretchJawAngle)
        j_msg = JointState()
        j_msg.header.stamp = timestamp
        j_msg.position = np.append(joint_angles, [jaw_angle]) + j_bias
        j_publisher.publish(j_msg)

        #publish ecm pose
        p_msg.header.stamp = timestamp
        p_publisher.publish(p_msg)
        j_ecm_publisher.publish()
        j_msg_ecm = JointState()
        j_msg_ecm.header.stamp = timestamp
        j_msg_ecm.position = ecm_j_angles
        j_ecm_publisher.publish(j_msg_ecm)
        
        if firstTime:
            time.sleep(0.5)
            firstTime = False
            continue

        #
        # Save all the data
        #

        #real hand-eye (redundantly saved)
        h_pos, h_quat = getCurrentHandEye()
        save_data = np.append(timestamp, h_pos)
        realHandeye_csv.writerow(np.append(save_data, vrep_to_transforms_quat(h_quat)).tolist())
        
        #real end-effector in cam frame
        e_pos, e_quat = psm1.getPoseAtHandle(psm1.EE_virtual_handle, left_cam.camera_handle,  initialize=False)
        save_data = np.append(timestamp, e_pos)
        realEndEffector_csv.writerow(np.append(save_data, vrep_to_transforms_quat(e_quat)).tolist())
        
        #real joint angles:
        joint_angles, jaw_angle = psm1.getJointAngles(ignoreError=True, initialize=False)
        save_data = np.append(timestamp, joint_angles)
        realJointAngles_csv.writerow(np.append(save_data, jaw_angle).tolist())
        
        #measured joint angles:
        measuredJointAngles_csv.writerow(np.append(timestamp, j_msg.position).tolist())

        #raw images
        cv2.imwrite(rawImageFilePath + "l_" +  str(timestamp.to_nsec()) + ".png", l_img)
        cv2.imwrite(rawImageFilePath + "r_" +  str(timestamp.to_nsec()) + ".png", r_img)
                                             
        #ECM pose
        ecm_pos, ecm_quat = ecm.getCurrentPose()
        save_data = np.append(timestamp, ecm_pos)
        realECMPose_csv.writerow(np.append(save_data, ecm_quat).tolist())
                                             
        #ECM real joint angles
        save_data = np.append(timestamp, ecm_noisy_j_angles)
        realJointAnglesECM_csv.writerow(save_data.tolist())
        
        #ECM measured joint angles
        save_data = np.append(timestamp, ecm_j_angles)
        measuredJointAnglesECM_csv.writerow(save_data.tolist())

        #save tracked joint angles
        global most_recent_tracked_joint
        while not haveNewJTrackedData:
            continue
        trackedJointAngles_csv.writerow(np.append(timestamp, most_recent_tracked_joint).tolist())
               
            
        #save tracked pose
        global most_recent_tracked_pose
        while not haveNewPTrackedData:
            continue
        save_data = np.append(timestamp, [most_recent_tracked_pose.position.x,
                                          most_recent_tracked_pose.position.y,
                                          most_recent_tracked_pose.position.z])
        save_data = np.append(save_data, [most_recent_tracked_pose.orientation.w, 
                                          most_recent_tracked_pose.orientation.x,
                                          most_recent_tracked_pose.orientation.y,
                                          most_recent_tracked_pose.orientation.z])
        trackedEndEffector_csv.writerow(save_data.tolist())
        
        # Save lumped error:
        listener.waitForTransform("/PSM1_base", "/cam0", timestamp, rospy.Duration(4.0))
        (trans, rot) = listener.lookupTransform("/PSM1_base", "/cam0", rospy.Time(0))
        save_data = np.append(timestamp, trans)
        save_data = np.append(save_data, rot[-1])
        save_data = np.append(save_data, rot[:3])
        trackedLumpedError_csv.writerow(save_data.tolist())
        
         #save rendered images!!!
        global most_recent_tracked_limg
        while not haveNewLTrackedData:
            continue
        cv2.imwrite(renderImageFilePath + "l_" +  str(timestamp.to_nsec()) + ".png", most_recent_tracked_limg)

        global most_recent_tracked_rimg
        while not haveNewRTrackedData:
            continue
        cv2.imwrite(renderImageFilePath + "r_" +  str(timestamp.to_nsec()) + ".png", most_recent_tracked_rimg)


        start_itr += 1
        if start_itr <= 20:
            continue

        loop_itr += 1
        if loop_itr >= 125:
            break
            
    p.terminate()


In [18]:
std_rot_ecm   = 0.01
std_prism_ecm = 0.0025

biasRot = 0.25 #deg
biasIns = 0.2 #cm
biasJaw = 0.01 #0 to 1 = open

cableStretchRot = 2#1 #deg per 90deg
cableStretchIns = 0.5#0.5 #cm per 20cm
cableStretchJaw = 0.05#0.05 #per 1 = open


handEyePosNois_std =  0.005#0.005#0.0025#0.001 #0.0025 m
handEyeRotNoise_std = 0.05#0.05# 0.02 radians



hand_eye_fp = "/home/arclab/vrep_dvrk_sim/dvrk_simulator/vrep_sims_for_tool_tracking/vrep_dataset_noisy_handeye.yaml"


runMovingArmAndECMExperiment(cableStretchRot, cableStretchIns, cableStretchJaw, 
                             biasRot, biasIns, biasJaw, 
                             std_rot_ecm, std_prism_ecm,
                             handEyePosNois_std, handEyeRotNoise_std, #for hand-eye noise
                             hand_eye_fp, "/home/arclab/vrep_dvrk_sim/dvrk_simulator/vrep_sims_for_tool_tracking/test/")


In [20]:
from shutil import copyfile

hand_eye_fp = "/home/arclab/vrep_dvrk_sim/dvrk_simulator/vrep_sims_for_tool_tracking/vrep_dataset_noisy_handeye.yaml"

std_rot_ecm   = [0, 0.01]
std_prism_ecm = [0, 0.0025]

cableStretchRot = [0, 2]#1 #deg per 90deg
cableStretchIns = [0, 0.5]#0.5 #cm per 20cm
cableStretchJaw = [0, 0.05]#0.05 #per 1 = open

biasRot = [0, 0.25]#0.25 #deg
biasIns = [0, 0.2]#0.1 #cm
biasJaw = [0, 0.01]#0.01 #0 to 1 = open

handEyePosNois_std =  [0, 0.005]#0.0025#0.001 #0.0025 m
handEyeRotNoise_std = [0, 0.05]# 0.02 radians

num_repeat_trials = 50


trials_to_run = [ (1,1, 0), (0,0,1), (1,1,1)]

# Naming convenction:
# --Tracking specific--
# h0, h1 = "hyand-eye" tracking off/on
# j1, j2, ... , j8 = first joint to track
# l0, l1 = lumped joint error prediction off/on
# --Environment specific--
# c0, c1 = cable stretch of/on
# b0, b1 = bias off/on
# n0, n1 = hand-eye noise off/on

#Total number of combinations 

name_convention = "h1_j8_l0"

save_parent_folder = "/home/arclab/vrep_dvrk_sim/dvrk_simulator/vrep_sims_for_tool_tracking/save_data_moving/" + name_convention + "/"
createDirectory(save_parent_folder)

copyfile("/home/arclab/vrep_dvrk_sim/dvrk_simulator/vrep_sims_for_tool_tracking/vrep_dataset_configure_filter_with_ecm.json",
         save_parent_folder + "vrep_dataset_configure_filter_with_ecm.json")

for i in reversed(range(len(cableStretchRot))):
    for j in reversed(range(len(biasRot))):
        for k in reversed(range(len(handEyePosNois_std))):
            
            if not ((i,j,k) in trials_to_run):
                continue
            
            name = name_convention + "_c{}_b{}_n{}".format(i,j,k)
            save_child_folder = save_parent_folder + name + "/"
            createDirectory(save_child_folder)

            for trial in range(num_repeat_trials):
                #Skip if it already is there!
                save_folder = save_child_folder + str(trial) + "/"
                if os.path.exists(save_folder):
                    continue

                print("Starting trial: {} for {}".format(trial, name))
                
                runMovingArmAndECMExperiment(cableStretchRot[i], cableStretchIns[i], cableStretchJaw[i], 
                             biasRot[j], biasIns[j], biasJaw[j], 
                             std_rot_ecm[i], std_prism_ecm[i],
                             handEyePosNois_std[k], handEyeRotNoise_std[k], #for hand-eye noise
                             hand_eye_fp, save_folder)


Starting trial: 0 for h1_j8_l0_c1_b1_n1
Starting trial: 1 for h1_j8_l0_c1_b1_n1
Starting trial: 2 for h1_j8_l0_c1_b1_n1
Starting trial: 3 for h1_j8_l0_c1_b1_n1
Starting trial: 4 for h1_j8_l0_c1_b1_n1
Starting trial: 5 for h1_j8_l0_c1_b1_n1
Starting trial: 6 for h1_j8_l0_c1_b1_n1
Starting trial: 7 for h1_j8_l0_c1_b1_n1
Starting trial: 8 for h1_j8_l0_c1_b1_n1
Starting trial: 9 for h1_j8_l0_c1_b1_n1
Starting trial: 10 for h1_j8_l0_c1_b1_n1
Starting trial: 11 for h1_j8_l0_c1_b1_n1
Starting trial: 12 for h1_j8_l0_c1_b1_n1
Starting trial: 13 for h1_j8_l0_c1_b1_n1
Starting trial: 14 for h1_j8_l0_c1_b1_n1
Starting trial: 15 for h1_j8_l0_c1_b1_n1
Starting trial: 16 for h1_j8_l0_c1_b1_n1
Starting trial: 17 for h1_j8_l0_c1_b1_n1
Starting trial: 18 for h1_j8_l0_c1_b1_n1
Starting trial: 19 for h1_j8_l0_c1_b1_n1
Starting trial: 20 for h1_j8_l0_c1_b1_n1
Starting trial: 21 for h1_j8_l0_c1_b1_n1
Starting trial: 22 for h1_j8_l0_c1_b1_n1
Starting trial: 23 for h1_j8_l0_c1_b1_n1
Starting trial: 24 for h1_

In [70]:
import subprocess
import time
import csv
import tf


e_pos, e_quat = psm1.getPoseAtHandle(psm1.EE_virtual_handle, left_cam.camera_handle,  initialize=True)


def runTestInitializationArmAndECMExperiment(cableStretchPer90, cableStretchInsertionPerCM, cableStretchJawAngle, 
                                             abs_max_bias_rot, abs_max_bias_ins, abs_max_bias_jaw, 
                                             std_rot_ecm, std_prism_ecm,
                                             std_pos_noise, std_rot_noise, #for hand-eye noise
                                             hand_eye_filepath):
    #save hand-eye
    starting_ecm_j_angles = [0., 0.2617994,  0., 0.]
    ecm.setJointAngles(starting_ecm_j_angles)

    vrep.simxSynchronousTrigger(clientID)
    vrep.simxGetPingTime(clientID)        
    vrep.simxSynchronousTrigger(clientID)
    vrep.simxGetPingTime(clientID)

    h_pos, h_quat = getCurrentHandEye()
    h_pos, h_quat = applyNormalNoiseToPose(h_pos, h_quat, std_pos_noise, std_rot_noise)
    ecm_pos, ecm_quat = ecm.getCurrentPose()
    writeHandEye(h_pos, h_quat, ecm_pos, ecm_quat, hand_eye_filepath)
    
    cmd = ["rosrun", "particle_filter", "particle_filter_node", 
           "/home/arclab/vrep_dvrk_sim/dvrk_simulator/vrep_sims_for_tool_tracking/vrep_dataset_configure_filter_with_ecm.json"]
    p = subprocess.Popen(cmd, stdout=subprocess.PIPE, stdin=subprocess.PIPE, stderr=subprocess.PIPE)
    time.sleep(4)
    
    std_of_random_position_traj = 0.0001 #in m, this is additive noise on trajectory
    max_rotation_about_random_axis_traj = 0.07 #in radians for trajectory, random walk!

    rate = rospy.Rate(100)

    #parameters for position trajectory
    xy_circle_amplitude = 0.025
    xy_circle_frequency = np.pi/180.0
    z_circle_amplitude = 0.01
    z_circle_frequency = np.pi/90.0
    jaw_rate = 1/25.0

    center_point  = [ 0.1405422 ,  0.03579695, -0.10973072];
    starting_quat = [-0.47013299, -0.26924748, -0.14768405,  0.827448  ]
    quat = starting_quat
    
    #Generate random bias values
    j_bias = np.random.uniform(-abs_max_bias_rot*np.pi/180.0, abs_max_bias_rot*np.pi/180.0, 2)
    j_bias = np.append(j_bias, np.random.uniform(-abs_max_bias_ins/100.0,abs_max_bias_ins/100.0,1))
    j_bias = np.append(j_bias, np.random.uniform(-abs_max_bias_rot*np.pi/180.0, abs_max_bias_rot*np.pi/180.0, 3))
    j_bias = np.append(j_bias, np.random.uniform(-abs_max_bias_jaw, abs_max_bias_jaw, 1))
    
    j_ecm_bias = np.random.uniform(-abs_max_bias_rot*np.pi/180.0, abs_max_bias_rot*np.pi/180.0, 2)
    j_ecm_bias = np.append(j_ecm_bias, np.random.uniform(-abs_max_bias_ins/100.0,abs_max_bias_ins/100.0,1))
    j_ecm_bias = np.append(j_ecm_bias, np.random.uniform(-abs_max_bias_rot*np.pi/180.0, abs_max_bias_rot*np.pi/180.0, 1))
                                             
    start_itr = 0.0
    loop_itr  = 0.0
    
    firstTime =  True
    while not rospy.is_shutdown():

        #Do the trajectory
        x = xy_circle_amplitude*np.cos(xy_circle_frequency*loop_itr)
        y = xy_circle_amplitude*np.sin(xy_circle_frequency*loop_itr)
        z = z_circle_amplitude*np.sin(z_circle_frequency*loop_itr)

        pos  = center_point + np.array([x, y, z])
        pos, quat = applyRandomNoiseToPose(pos, quat, std_of_random_position_traj, max_rotation_about_random_axis_traj)

        #Move jaw!
        jaw  = math.fmod(jaw_rate*loop_itr, 2.0)
        if jaw > 1.0:
            jaw = 2.0 - jaw

        psm1.setPoseAtEE(pos, quat, 1 - jaw)
                                             
        #Do the trajectory for the ecm

        #first get the non-noisy pose which is published to the ros world later  
        ecm_j_angles = starting_ecm_j_angles + np.array([0.2*np.sin(1.5*xy_circle_frequency*loop_itr),
                                                         -0.1*np.sin(xy_circle_frequency*loop_itr),
                                                         -0.05*np.sin(xy_circle_frequency*loop_itr),
                                                         0.3*np.sin(xy_circle_frequency*loop_itr)])
                                             
        ecm.setJointAngles(ecm_j_angles)
        vrep.simxSynchronousTrigger(clientID)
        vrep.simxGetPingTime(clientID)        

        pos, quat = ecm.getCurrentPose()
        p_msg = PoseStamped()
        p_msg.pose.position.x = pos[0]
        p_msg.pose.position.y = pos[1]
        p_msg.pose.position.z = pos[2]
        p_msg.pose.orientation.x = quat[0]
        p_msg.pose.orientation.y = quat[1]
        p_msg.pose.orientation.z = quat[2]
        p_msg.pose.orientation.w = quat[3]

        #now set the noisy joint angles which give the image
        ecm_noisy_j_angles = applyNoisyJointAnglesECM(ecm_j_angles, std_rot_ecm, std_prism_ecm) + j_ecm_bias
        ecm.setJointAngles(ecm_noisy_j_angles)
        
        global haveNewPTrackedData
        haveNewPTrackedData = False
        
        vrep.simxSynchronousTrigger(clientID)
        vrep.simxGetPingTime(clientID)   
        
        #Get timestamp and make the same for all data so it is sync'ed
        timestamp = rospy.get_rostime()

        #Publish images
        l_img, r_img = getStereoImagePairs()
        l_msg = bridge.cv2_to_imgmsg(l_img , encoding ="rgb8");
        l_msg.header.stamp = timestamp
        r_msg = bridge.cv2_to_imgmsg(r_img , encoding ="rgb8");
        r_msg.header.stamp = timestamp

        l_image_pub.publish(l_msg)
        r_image_pub.publish(r_msg)

        #Publish joint angles
        joint_angles, jaw_angle = getStretchedJointAngles(cableStretchPer90, 
                                                          cableStretchInsertionPerCM,
                                                          cableStretchJawAngle)
        j_msg = JointState()
        j_msg.header.stamp = timestamp
        j_msg.position = np.append(joint_angles, [jaw_angle]) + j_bias
        j_publisher.publish(j_msg)

        #publish ecm pose
        p_msg.header.stamp = timestamp
        p_publisher.publish(p_msg)
        j_ecm_publisher.publish()
        j_msg_ecm = JointState()
        j_msg_ecm.header.stamp = timestamp
        j_msg_ecm.position = ecm_j_angles
        j_ecm_publisher.publish(j_msg_ecm)
        
        if firstTime:
            time.sleep(0.5)
            firstTime = False
            continue
            
        #real end-effector in cam frame
        e_pos, e_quat = psm1.getPoseAtHandle(psm1.EE_virtual_handle, left_cam.camera_handle,  initialize=False)                       
        e_quat = vrep_to_transforms_quat(e_quat)    
        T_extra = np.array([[-1, 0, 0, 0],
                            [0, 1, 0, 0],
                            [0, 0,  -1, 0],
                            [0,  0,  0, 1]])
        
        T_real = np.dot(posQuatToMatrix(e_pos, e_quat), T_extra)        
        e_pos = T_real[:3, 3]
        
        # tracked pose
        global most_recent_tracked_pose
        while not haveNewPTrackedData:
            continue
            
        t_pos  = np.array([most_recent_tracked_pose.position.x, 
                           most_recent_tracked_pose.position.y, 
                           most_recent_tracked_pose.position.z])
        
        t_quat = np.array([most_recent_tracked_pose.orientation.w, 
                           most_recent_tracked_pose.orientation.x,
                           most_recent_tracked_pose.orientation.y,
                           most_recent_tracked_pose.orientation.z])
        dist_error = np.linalg.norm(t_pos - e_pos)
        
        e_euler = transforms3d.euler.mat2euler(T_real[:3,:3])
        ori_error  = np.linalg.norm(np.array(e_euler) - np.array(transforms3d.euler.quat2euler(t_quat)))


        start_itr += 1
        if start_itr >= 20:
            break
            
    p.terminate()
    return dist_error, ori_error

In [75]:
hand_eye_fp = "/home/arclab/vrep_dvrk_sim/dvrk_simulator/vrep_sims_for_tool_tracking/vrep_dataset_noisy_handeye.yaml"

std_rot_ecm   = 0.01
std_prism_ecm = 0.0015
    

cableStretchRot = 1#1 #deg per 90deg
cableStretchIns = 0.5#0.5#0.5 #cm per 20cm
cableStretchJaw = 0.05#0.05 #per 1 = open

biasRot = 0.25 #deg
biasIns = 0.2#0.2 #cm
biasJaw = 0.01 #0 to 1 = open

handEyePosNois_std = 0.005 # mm
handEyeRotNoise_std = 0.05 # 0.02 radians

num_itr = 50

d_error = np.zeros(num_itr)
o_error = np.zeros(num_itr)

for i in range(num_itr):
    d_error[i], o_error[i] = runTestInitializationArmAndECMExperiment(cableStretchRot, cableStretchIns, cableStretchJaw, 
                                                                     biasRot, biasIns, biasJaw, 
                                                                     std_rot_ecm, std_prism_ecm,
                                                                     handEyePosNois_std, handEyeRotNoise_std, 
                                                                     hand_eye_fp)
    d_error[i] *= 1000.0
    o_error[i] *= 180.0/np.pi
    print("{} Error d = {}, Error o = {}".format(i, d_error[i], o_error[i]))

print("Distance error  : {} +/- {} with max {}".format(d_error[:i+1].mean(), d_error[:i+1].std(), d_error[:i+1].max()))
print("Orienation error: {} +/- {} with max {}".format(o_error[:i+1].mean(), o_error[:i+1].std(), o_error[:i+1].max()))

0 Error d = 2.96965407949, Error o = 5.31471030031
1 Error d = 13.511952787, Error o = 3.16896011978
2 Error d = 0.496735876513, Error o = 5.4242439557
3 Error d = 4.72511035371, Error o = 2.82230419342
4 Error d = 1.17868829654, Error o = 3.07055537421
5 Error d = 0.821303458618, Error o = 4.40428904455
6 Error d = 5.2383870016, Error o = 6.03270983653
7 Error d = 0.839768944185, Error o = 1.84217537988
8 Error d = 4.48790487607, Error o = 3.5132000972
9 Error d = 3.91145914832, Error o = 4.56969517437
10 Error d = 0.818197314379, Error o = 2.53181858149
11 Error d = 6.12687947602, Error o = 4.42985859553
12 Error d = 6.05045380489, Error o = 3.88840810976
13 Error d = 4.84665841518, Error o = 0.837226993037
14 Error d = 1.1984549522, Error o = 3.76944822095
15 Error d = 2.53241948982, Error o = 5.26001742323
16 Error d = 6.4068503225, Error o = 3.9388989755
17 Error d = 3.70696713771, Error o = 3.95367582489
18 Error d = 2.84480304913, Error o = 6.73708665985
19 Error d = 8.197370354


Distance error  : 3.69363573165 +/- 1.5296423293 with max 6.64174671029
Orienation error: 2.9765091081 +/- 0.691636358242 with max 4.01297214697

No lumped error with 5, 1.0, 0.01
Distance error  : 2.53235495209 +/- 1.58618594081 with max 5.809590102
Orienation error: 3.3296414051 +/- 1.2038611327 with max 5.59308444028



Lumped error calc with 5, 2.0, 0.02 for scale, sigma_t, and sigma_r


Lumped error calc with 5, 2.0, 0.01 for scale, sigma_t, and sigma_r
Distance error  : 7.34036114649 +/- 5.12099443903 with max 19.8896239718
Orienation error: 4.69915720534 +/- 1.25253041033 with max 6.89017052801

Lumped error calc with 20, 0.5, 0.01 for scale, sigma_t, and sigma_r
Distance error  : 6.30505059259 +/- 5.2249912033 with max 15.4528597818
Orienation error: 8.3714336771 +/- 5.09356951588 with max 16.2614062796

This is tracking only lumped error with 20, 0.5, 0.01 for scale, sigma_t, and sigma_r
Distance error  : 7.7232472287 +/- 7.9995704229 with max 26.9679491869
Orienation error: 7.24919995577 +/- 3.20047095002 with max 12.136384228